# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here
scaler = StandardScaler()

# Scale X_train and X_test using StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Ensure X_train and X_test are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train = pd.DataFrame(X_train_scaled, 
                                   columns=X.columns, 
                                   index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, 
                                  columns=X.columns, 
                                  index=X_test.index)

In [4]:
# Create a LinearRegression model and fit it on scaled training data
lr = LinearRegression()
lr.fit(X_train, y_train)
# Calculate a baseline r-squared score on training data
baseline = lr.score(X_train, y_train)
baseline

0.7868344817421309

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [5]:
# Set up data structure
interactions = []

# Find combinations of columns and loop over them
for combo in combinations(X_train.columns,2):
    # Make copies of X_train and X_test
    train = X_train.copy()
    test = X_test.copy()
    
    # Add interaction term to data
    train['interaction'] = train[combo[0]]*train[combo[1]]
    test['interaction'] = test[combo[0]]*test[combo[1]]
    
    # Find r-squared score (fit on training data, evaluate on test data)
    lr = LinearRegression()
    lr.fit(train,y_train)
    R2 = lr.score(test,y_test)
    
    # Append to data structure
    interactions.append([combo[0],combo[1],R2])
    
# Sort and subset the data structure to find the top 7
inter_df = pd.DataFrame(interactions,columns=['feat_1','feat_2','r_squared'])
top_7_df = inter_df.sort_values('r_squared',ascending=False).iloc[:7]
top_7_df

,feat_1,feat_2,r_squared
3,LotArea,1stFlrSF,0.721111
2,LotArea,TotalBsmtSF,0.707165
5,LotArea,GrLivArea,0.669098
8,LotArea,Fireplaces,0.652970
36,2ndFlrSF,TotRmsAbvGrd,0.647299
17,OverallCond,TotalBsmtSF,0.642902
12,OverallQual,2ndFlrSF,0.642232


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_test` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [6]:
# Loop over top 7 interactions
for col1, col2 in top_7_df.iloc[:,:2].itertuples(index=False, name=None):
    # Extract column names from data structure
    
    # Construct new column name
    name = str(col1)+'_'+str(col2)
    # Add new column to X_train and X_test
    X_train[name] = X_train[col1]*X_train[col2]
    X_test[name] = X_test[col1]*X_test[col2]
X_train.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
518,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
236,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
38,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
1034,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
520,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [7]:
# Set up data structure
poly_list = []
# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
    for deg in (2, 3, 4):
        # Make a copy of X_train and X_test
        train = X_train.copy().reset_index()
        test = X_test.copy().reset_index()

        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree=deg, include_bias=False)

        # Fit polynomial to column and transform column
        poly_feat_train = pd.DataFrame(poly.fit_transform(train[[col]]))
        poly_feat_test = pd.DataFrame(poly.transform(test[[col]]))
        
        # Add polynomial to data
        poly_train = pd.concat([train.drop(col, axis=1), poly_feat_train], axis=1)
        poly_test = pd.concat([test.drop(col, axis=1), poly_feat_test], axis=1)
        
        # Find r-squared score
        lr = LinearRegression()
        lr.fit(poly_train, y_train)
        R2 = lr.score(poly_test, y_test)
        
        # Append to data structure
        poly_list.append(((col, deg, R2)))

# Sort and subset the data structure to find the top 7
poly_df = pd.DataFrame(poly_list, columns=['col_name', 'degree', 'r_squared'])
sorted_poly = poly_df.sort_values('r_squared', ascending=False)
num = 7
while len(sorted_poly['col_name'].iloc[:num].unique()) < 7:
    num += 1

top_poly = sorted_poly.iloc[:num]
top_poly

,col_name,degree,r_squared
19,GrLivArea,3,0.827957
49,OverallQual_2ndFlrSF,3,0.821185
41,LotArea_Fireplaces,4,0.811689
40,LotArea_Fireplaces,3,0.811442
4,OverallQual,3,0.805263
48,OverallQual_2ndFlrSF,2,0.805042
5,OverallQual,4,0.802168
3,OverallQual,2,0.801164
2,LotArea,4,0.799193
16,2ndFlrSF,3,0.775607


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [8]:
# Filter out duplicates
col_names = []
poly_terms = []
for row in top_poly.iterrows():
    col = row[1][0]
    deg = row[1][1]
    if col not in col_names:
        col_names.append(col)
        poly_terms.append([col,deg])
poly_terms

[['GrLivArea', 3],
 ['OverallQual_2ndFlrSF', 3],
 ['LotArea_Fireplaces', 4],
 ['OverallQual', 3],
 ['LotArea', 4],
 ['2ndFlrSF', 3],
 ['2ndFlrSF_TotRmsAbvGrd', 3]]

In [9]:
# Loop over remaining results
for col, deg in poly_terms:

    # Create polynomial column names
    names = []
    for i in range(deg + 1):
        if i > 0:
            names.append(col + '^' + str(i))

    # Create polynomial terms
    poly = PolynomialFeatures(degree=deg, include_bias=False)
    poly_feat_train = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                                   columns=names,
                                   index=X_train.index).iloc[:, 1:] # first degree column is a duplicate
    poly_feat_test = pd.DataFrame(poly.transform(X_test[[col]]),
                                  columns=names,
                                  index=X_test.index).iloc[:, 1:]

    # Concat new polynomials to X_train and X_test
    X_train = pd.concat([X_train, poly_feat_train], axis=1)
    X_test = pd.concat([X_test, poly_feat_test], axis=1)

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [10]:
print(X_train.columns==X_test.columns)
display(X_test.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True]


Index(['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces',
       'LotArea_1stFlrSF', 'LotArea_TotalBsmtSF', 'LotArea_GrLivArea',
       'LotArea_Fireplaces', '2ndFlrSF_TotRmsAbvGrd',
       'OverallCond_TotalBsmtSF', 'OverallQual_2ndFlrSF', 'GrLivArea^2',
       'GrLivArea^3', 'OverallQual_2ndFlrSF^2', 'OverallQual_2ndFlrSF^3',
       'LotArea_Fireplaces^2', 'LotArea_Fireplaces^3', 'LotArea_Fireplaces^4',
       'OverallQual^2', 'OverallQual^3', 'LotArea^2', 'LotArea^3', 'LotArea^4',
       '2ndFlrSF^2', '2ndFlrSF^3', '2ndFlrSF_TotRmsAbvGrd^2',
       '2ndFlrSF_TotRmsAbvGrd^3'],
      dtype='object')

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and test data.

In [11]:
# Your code here
lr = LinearRegression()
lr.fit(X_train,y_train)
R2_train_full = lr.score(X_train,y_train)
R2_test_full = lr.score(X_test,y_test)
print(f'Training R-Squared:{R2_train_full}')
print(f'Test R-Squared:{R2_test_full}')



Training R-Squared:0.8662571545061544
Test R-Squared:-53.74748395708928


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and test $R^2$ score and how many features remain.

In [12]:
# Your code here
rfe_list = []
num_features = X_train.shape[1]
for num in range(num_features):
    rfe = RFE(LinearRegression(),n_features_to_select=num+1)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_test = rfe.transform(X_test)
    lr = LinearRegression()
    lr.fit(X_rfe_train, y_train)

    train_r2 = lr.score(X_rfe_train, y_train)
    test_r2 = lr.score(X_rfe_test, y_test)
    num_feat = X_rfe_train.shape[1]
    rfe_list.append((train_r2,test_r2,num_feat))
rfe_df = pd.DataFrame(rfe_list,columns=['train_r2','test_r2','num_feat'])
rfe_df

,train_r2,test_r2,num_feat
0,0.626357,0.588350,1
1,0.701676,0.606605,2
2,0.748018,0.644873,3
3,0.761338,0.614568,4
4,0.776040,0.635298,5
5,0.802565,0.645120,6
6,0.811519,0.648715,7
7,0.817568,0.771919,8
8,0.821070,0.773950,9
9,0.823568,0.769994,10


In [13]:
# Seeing how many features produces the best results
rfe_df.sort_values('test_r2',ascending=False).head()

,train_r2,test_r2,num_feat
13,0.835579,0.834598,14
12,0.835567,0.834553,13
11,0.835025,0.833924,12
14,0.846323,0.815012,15
10,0.831086,0.797074,11


In [14]:
rfe = RFE(LinearRegression(),n_features_to_select=14)
X_rfe_train = rfe.fit_transform(X_train, y_train)
X_rfe_test = rfe.transform(X_test)
lr = LinearRegression()
lr.fit(X_rfe_train, y_train)
X_train.iloc[:,rfe.support_].columns



Index(['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces',
       'LotArea_1stFlrSF', 'GrLivArea^2', 'OverallQual^2', '2ndFlrSF^2',
       '2ndFlrSF^3'],
      dtype='object')

In [15]:
lasso = Lasso(alpha=1000000)
lasso.fit(X_train,y_train)
sum(abs(lasso.coef_) > 10**(-10))

3

Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [16]:
# Your code here
lasso_list = []
for num in [0.1, 1, 10, 100, 1000, 10000, 100000, 1000000]:
    lasso = Lasso(alpha=num)
    lasso.fit(X_train,y_train)
    
    train_r2 = lasso.score(X_train, y_train)
    test_r2 = lasso.score(X_test, y_test)
    num_feats = sum(abs(lasso.coef_) > 10**(-10))
    
    lasso_list.append((train_r2,test_r2,num,num_feats))
lasso_df = pd.DataFrame(lasso_list,columns=['train_r2','test_r2','alpha','num_feat'])
lasso_df

,train_r2,test_r2,alpha,num_feat
0,0.861986,0.567678,0.1,33
1,0.861985,0.567726,1.0,33
2,0.861965,0.568195,10.0,33
3,0.861725,0.567790,100.0,32
4,0.855464,0.637995,1000.0,22
5,0.789476,0.789881,10000.0,17
6,0.298107,0.304605,100000.0,5
7,0.066689,0.040144,1000000.0,3


Compare the results. Which features would you choose to use?

In [17]:
# Your written answer here

"""
The best results came from doing the RFE with 14 features. Which include:
'LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF',
'2ndFlrSF', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces',
'LotArea_1stFlrSF', 'GrLivArea^2', 'OverallQual^2', '2ndFlrSF^2',
'2ndFlrSF^3'
"""


"\nThe best results came from doing the RFE with 14 features. Which include:\n'LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF',\n'2ndFlrSF', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces',\n'LotArea_1stFlrSF', 'GrLivArea^2', 'OverallQual^2', '2ndFlrSF^2',\n'2ndFlrSF^3'\n"

## Evaluate Final Model on Validation Data

### Data Preparation

At the start of this lab, we created `X_val` and `y_val`. Prepare `X_val` the same way that `X_train` and `X_test` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [18]:
# Scale
X_val_scaled = scaler.transform(X_val)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns, index=X_val.index)

# Adding interactions
for col1, col2 in top_7_df.iloc[:,:2].itertuples(index=False, name=None):
    name = str(col1)+'_'+str(col2)
    X_val[name] = X_val[col1]*X_val[col2]

## Adding polynomial terms
for col, deg in poly_terms:
    names = []
    for i in range(deg + 1):
        if i > 0:
            names.append(col + '^' + str(i))
    poly = PolynomialFeatures(degree=deg, include_bias=False)
    poly.fit_transform(X_train[[col]])
    poly_feat_val = pd.DataFrame(poly.transform(X_val[[col]]),columns=names,index=X_val.index).iloc[:, 1:]
    X_val = pd.concat([X_val, poly_feat_val], axis=1)

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + test set, then find R-Squared and MSE values for the validation set.

In [19]:
X_train_test = pd.concat([X_train,X_test])   
y_train_test = pd.concat([y_train,y_test]) 

rfe = RFE(LinearRegression(),n_features_to_select=14)

X_rfe_train_test = rfe.fit_transform(X_train_test, y_train_test)
X_rfe_val = rfe.transform(X_val)
lr = LinearRegression()
lr.fit(X_train_test, y_train_test)

final_r2 = lr.score(X_val,y_val)
final_mse = mean_squared_error(y_val,lr.predict(X_val))

print(f'Final R-Squared:{final_r2}')
print(f'Final MSE:{final_mse}')

Final R-Squared:0.8544223741704797
Final MSE:1019816260.0416049


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.